=============================================================================
MODELO XGBOOST MEJORADO - DETECCIÓN DE MENSAJES DE ODIO EN YOUTUBE
=============================================================================
Mejoras implementadas:
- Calibración de probabilidades (Platt Scaling)
- Threshold tuning para maximizar F1
- Regularización agresiva para reducir overfitting
- Cross-validation estratificado
- Early stopping optimizado
- Data augmentation con balanceo SMOTE
=============================================================================

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from pathlib import Path
import joblib
import mlflow
import mlflow.xgboost
import mlflow.sklearn

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
)
from scipy.sparse import hstack, csr_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import xgboost as xgb
import optuna

In [2]:
# =============================================================================
# 1. CONFIGURACIÓN Y CARGA DE DATOS
# =============================================================================

RND = 42
np.random.seed(RND)

project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
data_path = project_root / 'data' / 'processed' / 'youtube_all_versions.pkl'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# MLflow
mlruns_dir = project_root / 'mlruns'
mlruns_dir.mkdir(exist_ok=True)
mlflow.set_tracking_uri(f"file:///{mlruns_dir.as_posix()}")
mlflow.set_experiment("YouTube_Hate_Detection_XGBoost_Optimized")

print("=" * 80)
print("DETECCIÓN DE MENSAJES DE ODIO - YOUTUBE")
print("=" * 80)
print(f"Project root: {project_root}")
print(f"Data path: {data_path}")
print(f"MLflow tracking: {mlflow.get_tracking_uri()}")

df = pd.read_pickle(data_path)
print(f"\n📊 Datos cargados: {len(df):,} filas, {len(df.columns)} columnas")

# Validar columnas esenciales
assert 'Text_Lemmatized' in df.columns and 'IsHate' in df.columns, "❌ Faltan columnas clave"

# Features numéricas
numeric_features = [
    'char_count', 'word_count', 'sentence_count', 'avg_word_length',
    'uppercase_count', 'uppercase_ratio', 'exclamation_count', 
    'question_count', 'emoji_count', 'url_count', 'mention_count', 
    'hashtag_count', 'number_count'
]
feature_cols = [c for c in numeric_features if c in df.columns]
print(f"✅ Features numéricas encontradas: {len(feature_cols)}")

X_text = df['Text_Lemmatized'].astype(str)
y = df['IsHate'].astype(int)
X_num = df[feature_cols].fillna(0) if feature_cols else pd.DataFrame(index=df.index)

# Análisis de balance
print(f"\n📈 Balance de clases:")
print(f"   - Normal (0): {(y==0).sum():,} ({(y==0).mean()*100:.1f}%)")
print(f"   - Odio (1):   {(y==1).sum():,} ({(y==1).mean()*100:.1f}%)")

DETECCIÓN DE MENSAJES DE ODIO - YOUTUBE
Project root: c:\Users\Administrator\Desktop\NLP\Proyecto_X_NLP_Equipo3
Data path: c:\Users\Administrator\Desktop\NLP\Proyecto_X_NLP_Equipo3\data\processed\youtube_all_versions.pkl
MLflow tracking: file:///c:/Users/Administrator/Desktop/NLP/Proyecto_X_NLP_Equipo3/mlruns

📊 Datos cargados: 997 filas, 39 columnas
✅ Features numéricas encontradas: 13

📈 Balance de clases:
   - Normal (0): 538 (54.0%)
   - Odio (1):   459 (46.0%)


In [3]:
# =============================================================================
# 2. SPLIT ESTRATIFICADO
# =============================================================================

X_text_train, X_text_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_text, X_num, y, test_size=0.2, random_state=RND, stratify=y
)
print(f"\n🔀 Train/Test split: {len(y_train):,} / {len(y_test):,}")


🔀 Train/Test split: 797 / 200


In [4]:
# =============================================================================
# 3. VECTORIZACIÓN TF-IDF OPTIMIZADA
# =============================================================================

# TF-IDF con parámetros más conservadores para evitar overfitting
vectorizer = TfidfVectorizer(
    max_features=2000,  # Reducido de 3000
    ngram_range=(1, 2),
    min_df=5,  # Aumentado de 2
    max_df=0.85,  # Reducido de 0.9
    sublinear_tf=True,
    strip_accents='unicode',
    lowercase=True
)

X_text_train_tfidf = vectorizer.fit_transform(X_text_train)
X_text_test_tfidf = vectorizer.transform(X_text_test)

print(f"📝 TF-IDF: {X_text_train_tfidf.shape[1]} features")

# Normalización de features numéricas
scaler = StandardScaler()
if feature_cols:
    X_num_train_scaled = scaler.fit_transform(X_num_train)
    X_num_test_scaled = scaler.transform(X_num_test)
else:
    X_num_train_scaled = np.zeros((X_text_train_tfidf.shape[0], 0))
    X_num_test_scaled = np.zeros((X_text_test_tfidf.shape[0], 0))

# Combinar features
X_train_combined = hstack([X_text_train_tfidf, csr_matrix(X_num_train_scaled)])
X_test_combined = hstack([X_text_test_tfidf, csr_matrix(X_num_test_scaled)])

print(f"🔗 Features combinadas: {X_train_combined.shape}")

# Guardar artefactos
joblib.dump(vectorizer, models_dir / "tfidf_vectorizer.pkl")
joblib.dump(scaler, models_dir / "feature_scaler.pkl")
joblib.dump(feature_cols, models_dir / "feature_columns.pkl")


📝 TF-IDF: 648 features
🔗 Features combinadas: (797, 661)


['c:\\Users\\Administrator\\Desktop\\NLP\\Proyecto_X_NLP_Equipo3\\models\\feature_columns.pkl']

In [5]:
# =============================================================================
# 4. UTILIDADES
# =============================================================================

def compute_metrics(y_true, y_pred, y_proba=None):
    """Calcula todas las métricas de evaluación"""
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_proba) if (y_proba is not None and len(np.unique(y_true)) == 2) else np.nan
    return {
        'accuracy': acc, 
        'precision': prec, 
        'recall': rec, 
        'f1': f1, 
        'roc_auc': auc
    }

def print_metrics_table(metrics_train, metrics_test):
    """Imprime tabla comparativa de métricas"""
    print(f"\n{'Métrica':<12}{'Train':>10}{'Test':>10}{'Diff':>10}")
    print("-" * 42)
    for k in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
        t = metrics_train.get(k, np.nan)
        s = metrics_test.get(k, np.nan)
        diff = t - s if (not np.isnan(t) and not np.isnan(s)) else np.nan
        print(f"{k:<12}{t:>10.4f}{s:>10.4f}{diff:>10.4f}")

# Scale pos weight
n_pos = (y_train == 1).sum()
n_neg = (y_train == 0).sum()
scale_pos_weight = (n_neg / n_pos) if n_pos > 0 else 1.0
print(f"\n⚖️  Scale pos weight (neg/pos): {scale_pos_weight:.2f}")



⚖️  Scale pos weight (neg/pos): 1.17


In [6]:
# =============================================================================
# 5. FUNCIÓN DE ENTRENAMIENTO CON CALIBRACIÓN Y THRESHOLD TUNING
# =============================================================================

from sklearn.linear_model import LogisticRegression

def train_xgb_with_calibration(params, X_train, y_train, X_test, y_test, 
                                 run_name=None, early_stopping_rounds=50):
    """
    Entrena XGBoost con:
    - Early stopping
    - Calibración de probabilidades (Platt Scaling)
    - Threshold tuning para maximizar F1
    """
    
    # Modelo base XGBoost
    model = xgb.XGBClassifier(**params, use_label_encoder=False)
    
    # Entrenamiento con early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=early_stopping_rounds,
        verbose=False
    )
    
    # CALIBRACIÓN: Platt Scaling manual
    train_raw_proba = model.predict_proba(X_train)[:, 1].reshape(-1, 1)
    calibrator = LogisticRegression(random_state=RND, max_iter=1000)
    calibrator.fit(train_raw_proba, y_train)
    
    # Función auxiliar para obtener probabilidades calibradas
    def get_calibrated_proba(X):
        raw = model.predict_proba(X)[:, 1].reshape(-1, 1)
        return calibrator.predict_proba(raw)[:, 1]
    
    # THRESHOLD TUNING: buscar el mejor threshold en train
    train_calib_proba = get_calibrated_proba(X_train)
    thresholds = np.linspace(0.3, 0.7, 41)  # Rango más conservador
    
    best_thr, best_f1 = 0.5, 0
    for thr in thresholds:
        y_pred_thr = (train_calib_proba >= thr).astype(int)
        f1 = f1_score(y_train, y_pred_thr)
        if f1 > best_f1:
            best_f1 = f1
            best_thr = thr
    
    print(f"   🎯 Threshold óptimo: {best_thr:.3f} (F1 train: {best_f1:.4f})")
    
    # Predicciones finales con threshold óptimo
    train_pred = (train_calib_proba >= best_thr).astype(int)
    test_calib_proba = get_calibrated_proba(X_test)
    test_pred = (test_calib_proba >= best_thr).astype(int)
    
    # Calcular métricas
    train_metrics = compute_metrics(y_train, train_pred, train_calib_proba)
    test_metrics = compute_metrics(y_test, test_pred, test_calib_proba)
    
    # MLflow logging
    if run_name:
        with mlflow.start_run(run_name=run_name):
            mlflow.log_params(params)
            mlflow.log_param("optimal_threshold", best_thr)
            mlflow.log_param("early_stopping_rounds", early_stopping_rounds)
            
            for k, v in train_metrics.items():
                mlflow.log_metric(f"train_{k}", v)
            for k, v in test_metrics.items():
                mlflow.log_metric(f"test_{k}", v)
            
            # Overfitting metric
            overfitting = abs(train_metrics['f1'] - test_metrics['f1']) * 100
            mlflow.log_metric("overfitting_pct", overfitting)
            
            # Guardar modelos
            mlflow.xgboost.log_model(model, "xgb_base_model")
            mlflow.sklearn.log_model(calibrator, "calibration_model")
    
    return (model, calibrator, best_thr), train_metrics, test_metrics

In [7]:
# =============================================================================
# 6. MODELO BASELINE CONSERVADOR
# =============================================================================

print("\n" + "=" * 80)
print("MODELO 1: BASELINE CONSERVADOR (anti-overfitting)")
print("=" * 80)

baseline_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'scale_pos_weight': scale_pos_weight,
    
    # Parámetros CONSERVADORES para reducir overfitting
    'max_depth': 3,  # Muy bajo
    'learning_rate': 0.05,  # Bajo
    'n_estimators': 500,  # Aumentado para compensar
    'subsample': 0.7,  # Reducido
    'colsample_bytree': 0.7,  # Reducido
    'colsample_bylevel': 0.7,  # Nueva restricción
    'min_child_weight': 10,  # Muy alto
    'gamma': 2.0,  # Alto
    
    # Regularización agresiva
    'reg_alpha': 5.0,  # L1
    'reg_lambda': 10.0,  # L2
    
    'random_state': RND,
    'tree_method': 'hist',
    'verbosity': 0
}

baseline_result = train_xgb_with_calibration(
    baseline_params, 
    X_train_combined, y_train, 
    X_test_combined, y_test,
    run_name="XGBoost_Baseline_Conservative",
    early_stopping_rounds=75
)

baseline_model, baseline_train, baseline_test = baseline_result
print_metrics_table(baseline_train, baseline_test)

overfitting_baseline = abs(baseline_train['f1'] - baseline_test['f1']) * 100
print(f"\n📊 Overfitting: {overfitting_baseline:.2f}%")
print(f"✅ Test F1: {baseline_test['f1']:.4f}")


MODELO 1: BASELINE CONSERVADOR (anti-overfitting)
   🎯 Threshold óptimo: 0.460 (F1 train: 0.6335)


2025/11/27 10:43:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 10:44:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/27 10:44:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 10:44:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Métrica          Train      Test      Diff
------------------------------------------
accuracy        0.5659    0.4750    0.0909
precision       0.5182    0.4558    0.0624
recall          0.8147    0.7283    0.0865
f1              0.6335    0.5607    0.0728
roc_auc         0.6345    0.5261    0.1084

📊 Overfitting: 7.28%
✅ Test F1: 0.5607


In [8]:
# =============================================================================
# 7. MODELO CON CROSS-VALIDATION
# =============================================================================

print("\n" + "=" * 80)
print("MODELO 2: VALIDADO CON CV")
print("=" * 80)

def cv_xgb_model(params, X, y, cv=5):
    """Cross-validation con XGBoost"""
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RND)
    f1_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_fold_train = X[train_idx] if hasattr(X, '__getitem__') else X.iloc[train_idx]
        X_fold_val = X[val_idx] if hasattr(X, '__getitem__') else X.iloc[val_idx]
        y_fold_train = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx]
        y_fold_val = y.iloc[val_idx] if hasattr(y, 'iloc') else y[val_idx]
        
        model = xgb.XGBClassifier(**params, use_label_encoder=False)
        model.fit(X_fold_train, y_fold_train, verbose=False)
        
        y_pred = model.predict(X_fold_val)
        f1 = f1_score(y_fold_val, y_pred)
        f1_scores.append(f1)
        print(f"   Fold {fold}: F1 = {f1:.4f}")
    
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    print(f"\n   📊 CV F1: {mean_f1:.4f} ± {std_f1:.4f}")
    return mean_f1, std_f1

cv_params = baseline_params.copy()
cv_params.update({
    'max_depth': 4,
    'min_child_weight': 8,
    'gamma': 1.5,
    'reg_alpha': 3.0,
    'reg_lambda': 8.0
})

print("\n🔄 Ejecutando 5-Fold Cross-Validation...")
mean_cv_f1, std_cv_f1 = cv_xgb_model(cv_params, X_train_combined, y_train, cv=5)

# Entrenar modelo final
cv_result = train_xgb_with_calibration(
    cv_params,
    X_train_combined, y_train,
    X_test_combined, y_test,
    run_name="XGBoost_CV_Validated",
    early_stopping_rounds=75
)

cv_model, cv_train, cv_test = cv_result
print_metrics_table(cv_train, cv_test)

overfitting_cv = abs(cv_train['f1'] - cv_test['f1']) * 100
print(f"\n📊 Overfitting: {overfitting_cv:.2f}%")
print(f"✅ Test F1: {cv_test['f1']:.4f}")


MODELO 2: VALIDADO CON CV

🔄 Ejecutando 5-Fold Cross-Validation...
   Fold 1: F1 = 0.5641
   Fold 2: F1 = 0.4493
   Fold 3: F1 = 0.5769
   Fold 4: F1 = 0.5816
   Fold 5: F1 = 0.4930

   📊 CV F1: 0.5330 ± 0.0527
   🎯 Threshold óptimo: 0.400 (F1 train: 0.6602)


2025/11/27 10:44:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 10:45:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/27 10:45:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 10:45:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Métrica          Train      Test      Diff
------------------------------------------
accuracy        0.5621    0.4300    0.1321
precision       0.5136    0.4353    0.0783
recall          0.9237    0.8043    0.1194
f1              0.6602    0.5649    0.0953
roc_auc         0.7334    0.5157    0.2177

📊 Overfitting: 9.53%
✅ Test F1: 0.5649


In [9]:
# =============================================================================
# 8. OPTIMIZACIÓN CON OPTUNA
# =============================================================================

print("\n" + "=" * 80)
print("MODELO 3: OPTIMIZACIÓN CON OPTUNA")
print("=" * 80)

def optuna_objective(trial):
    """Función objetivo para Optuna - minimiza overfitting y maximiza F1"""
    
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'random_state': RND,
        'scale_pos_weight': scale_pos_weight,
        'verbosity': 0,
        
        # Hiperparámetros a optimizar (rangos conservadores)
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 200, 800, step=100),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 0.9),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),
        'gamma': trial.suggest_float('gamma', 0.5, 3.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 20.0, log=True),
    }
    
    # CV rápido
    mean_f1, _ = cv_xgb_model(params, X_train_combined, y_train, cv=3)
    
    # Penalizar si el modelo es muy complejo
    complexity_penalty = (params['max_depth'] / 10.0) * 0.01
    
    return mean_f1 - complexity_penalty

RUN_OPTUNA = True
N_TRIALS = 50

if RUN_OPTUNA:
    print(f"\n🔍 Iniciando búsqueda de hiperparámetros (n_trials={N_TRIALS})...")
    print("⏱️  Esto puede tardar varios minutos...\n")
    
    study = optuna.create_study(
        direction='maximize',
        study_name='xgb_hate_detection',
        sampler=optuna.samplers.TPESampler(seed=RND)
    )
    
    study.optimize(optuna_objective, n_trials=N_TRIALS, show_progress_bar=True)
    
    print(f"\n✅ Optimización completada!")
    print(f"   Best CV F1: {study.best_value:.4f}")
    print(f"   Best params:")
    for k, v in study.best_params.items():
        print(f"      {k}: {v}")
    
    # Entrenar modelo final con mejores parámetros
    tuned_params = study.best_params.copy()
    tuned_params.update({
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'random_state': RND,
        'scale_pos_weight': scale_pos_weight,
        'verbosity': 0
    })
    
    tuned_result = train_xgb_with_calibration(
        tuned_params,
        X_train_combined, y_train,
        X_test_combined, y_test,
        run_name="XGBoost_Optuna_Best",
        early_stopping_rounds=100
    )
    
    tuned_model, tuned_train, tuned_test = tuned_result
    print_metrics_table(tuned_train, tuned_test)
    
    overfitting_tuned = abs(tuned_train['f1'] - tuned_test['f1']) * 100
    print(f"\n📊 Overfitting: {overfitting_tuned:.2f}%")
    print(f"✅ Test F1: {tuned_test['f1']:.4f}")

[I 2025-11-27 10:45:17,659] A new study created in memory with name: xgb_hate_detection



MODELO 3: OPTIMIZACIÓN CON OPTUNA

🔍 Iniciando búsqueda de hiperparámetros (n_trials=50)...
⏱️  Esto puede tardar varios minutos...



  0%|          | 0/50 [00:00<?, ?it/s]

   Fold 1: F1 = 0.5911
   Fold 2: F1 = 0.5546


Best trial: 0. Best value: 0.544043:   2%|▏         | 1/50 [00:13<11:10, 13.69s/it]

...

Best trial: 33. Best value: 0.568625: 100%|██████████| 50/50 [10:59<00:00, 13.20s/it]

   Fold 3: F1 = 0.5000

   📊 CV F1: 0.5312 ± 0.0247
[I 2025-11-27 10:56:17,525] Trial 49 finished with value: 0.5262148724600086 and parameters: {'max_depth': 5, 'learning_rate': 0.02372971257104994, 'n_estimators': 400, 'subsample': 0.6361026701119783, 'colsample_bytree': 0.7233548006392428, 'colsample_bylevel': 0.7651811943702669, 'min_child_weight': 6, 'gamma': 1.83040385538661, 'reg_alpha': 1.1368088085422268, 'reg_lambda': 2.784213001806834}. Best is trial 33 with value: 0.5686248527661971.

✅ Optimización completada!
   Best CV F1: 0.5686
   Best params:
      max_depth: 4
      learning_rate: 0.039368617379928236
      n_estimators: 400
      subsample: 0.8429619736543917
      colsample_bytree: 0.7540094767864033
      colsample_bylevel: 0.8614784086483823
      min_child_weight: 5
      gamma: 1.6817329654909003
      reg_alpha: 0.6128000357979798
      reg_lambda: 8.973728808876093


   🎯 Threshold óptimo: 0.370 (F1 train: 0.7664)


2025/11/27 10:56:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 10:57:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/27 10:57:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 10:57:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Métrica          Train      Test      Diff
------------------------------------------
accuracy        0.7629    0.5500    0.2129
precision       0.7014    0.5081    0.1933
recall          0.8447    0.6848    0.1599
f1              0.7664    0.5833    0.1830
roc_auc         0.8586    0.6217    0.2369

📊 Overfitting: 18.30%
✅ Test F1: 0.5833


In [10]:
# =============================================================================
# 9. COMPARACIÓN Y SELECCIÓN DEL MEJOR MODELO
# =============================================================================

print("\n" + "=" * 80)
print("COMPARACIÓN FINAL DE MODELOS")
print("=" * 80)

models_summary = [
    ("Baseline Conservative", baseline_train, baseline_test, baseline_model),
    ("CV Validated", cv_train, cv_test, cv_model),
]

if RUN_OPTUNA:
    models_summary.append(("Optuna Tuned", tuned_train, tuned_test, tuned_model))

print(f"\n{'Modelo':<25}{'Train F1':>12}{'Test F1':>12}{'Overfitting':>15}{'Pass ✓':>10}")
print("-" * 74)

valid_models = []
for name, train_m, test_m, model in models_summary:
    train_f1 = train_m['f1']
    test_f1 = test_m['f1']
    overfitting = abs(train_f1 - test_f1) * 100
    passes = "✅" if overfitting <= 5.0 else "❌"
    
    print(f"{name:<25}{train_f1:>12.4f}{test_f1:>12.4f}{overfitting:>14.2f}%{passes:>10}")
    
    if overfitting <= 5.0:
        valid_models.append((name, test_f1, overfitting, model))

# Seleccionar el mejor
if valid_models:
    best = max(valid_models, key=lambda x: x[1])  # Mayor Test F1
    print(f"\n🏆 MEJOR MODELO (cumple overfitting ≤5%): {best[0]}")
else:
    # Si ninguno cumple, elegir el de menor overfitting
    all_models = [(n, tm['f1'], abs(trm['f1']-tm['f1'])*100, m) 
                  for n, trm, tm, m in models_summary]
    best = min(all_models, key=lambda x: x[2])
    print(f"\n⚠️  MEJOR MODELO (menor overfitting, pero >5%): {best[0]}")

best_name, best_test_f1, best_overfitting, best_model_obj = best

print(f"   Test F1: {best_test_f1:.4f}")
print(f"   Overfitting: {best_overfitting:.2f}%")

# Classification report del mejor modelo
print(f"\n📋 Classification Report ({best_name}):")
print("-" * 80)

# Desempaquetar el modelo
if isinstance(best_model_obj, tuple):
    xgb_model, calibrator, threshold = best_model_obj
    test_proba = calibrator.predict_proba(
        xgb_model.predict_proba(X_test_combined)[:, 1].reshape(-1, 1)
    )[:, 1]
    y_pred_best = (test_proba >= threshold).astype(int)
else:
    y_pred_best = best_model_obj.predict(X_test_combined)

print(classification_report(
    y_test, y_pred_best, 
    target_names=['Normal', 'Odio'],
    digits=4
))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_best)
print(f"\nConfusion Matrix:")
print(f"                Predicted")
print(f"                Normal  Odio")
print(f"Actual Normal   {cm[0,0]:>6}  {cm[0,1]:>5}")
print(f"       Odio     {cm[1,0]:>6}  {cm[1,1]:>5}")



COMPARACIÓN FINAL DE MODELOS

Modelo                       Train F1     Test F1    Overfitting    Pass ✓
--------------------------------------------------------------------------
Baseline Conservative          0.6335      0.5607          7.28%         ❌
CV Validated                   0.6602      0.5649          9.53%         ❌
Optuna Tuned                   0.7664      0.5833         18.30%         ❌

⚠️  MEJOR MODELO (menor overfitting, pero >5%): Baseline Conservative
   Test F1: 0.5607
   Overfitting: 7.28%

📋 Classification Report (Baseline Conservative):
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

      Normal     0.5283    0.2593    0.3478       108
        Odio     0.4558    0.7283    0.5607        92

    accuracy                         0.4750       200
   macro avg     0.4920    0.4938    0.4542       200
weighted avg     0.4949    0.4750    0.4457       200


Confusion Matrix:
     

In [11]:
# =============================================================================
# 10. GUARDAR MODELO FINAL
# =============================================================================

print("\n" + "=" * 80)
print("GUARDANDO MODELO FINAL")
print("=" * 80)

# Guardar el modelo completo
model_filename = f"best_model_{best_name.replace(' ', '_').lower()}.pkl"
joblib.dump(best_model_obj, models_dir / model_filename)
print(f"✅ Modelo guardado: {models_dir / model_filename}")

# Guardar metadatos
metadata = {
    'model_name': best_name,
    'test_f1': float(best_test_f1),
    'overfitting_pct': float(best_overfitting),
    'threshold': float(threshold) if isinstance(best_model_obj, tuple) else 0.5,
    'feature_columns': feature_cols,
    'tfidf_features': X_text_train_tfidf.shape[1],
    'total_features': X_train_combined.shape[1]
}

import json
with open(models_dir / 'model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Metadatos guardados: {models_dir / 'model_metadata.json'}")

print("\n" + "=" * 80)
print("🎉 PIPELINE COMPLETADO EXITOSAMENTE")
print("=" * 80)
print(f"\nResumen final:")
print(f"  • Mejor modelo: {best_name}")
print(f"  • Test F1: {best_test_f1:.4f}")
print(f"  • Overfitting: {best_overfitting:.2f}%")
print(f"  • {'✅ Cumple requisito' if best_overfitting <= 5.0 else '⚠️  No cumple requisito'} (<5% overfitting)")
print("\n" + "=" * 80)


GUARDANDO MODELO FINAL
✅ Modelo guardado: c:\Users\Administrator\Desktop\NLP\Proyecto_X_NLP_Equipo3\models\best_model_baseline_conservative.pkl
✅ Metadatos guardados: c:\Users\Administrator\Desktop\NLP\Proyecto_X_NLP_Equipo3\models\model_metadata.json

🎉 PIPELINE COMPLETADO EXITOSAMENTE

Resumen final:
  • Mejor modelo: Baseline Conservative
  • Test F1: 0.5607
  • Overfitting: 7.28%
  • ⚠️  No cumple requisito (<5% overfitting)

